In [1]:
import pymysql
host = '192.168.50.17'
port = 3307
username = 'bot'
password = 'Ahdtlf@01!'
conn = pymysql.connect(host=host, user=username, passwd=password, port=port, use_unicode=True, charset='utf8')
cursor = conn.cursor()


In [5]:

from pyspark.sql import SparkSession
from pykis import *
from itertools import chain 
import pandas as pd 
import numpy as np 
# from library import mariadb
# from library.trading_algorithms import EAVG, EAVG_vol
from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta
import sys 
import os 
package_path = '/root/project/de-2024/jobs'
sys.path.append(package_path)
import cf

In [2]:
kis = PyKis(           
            appkey= cf.appkey
            ,appsecret= cf.appsecret
            ,virtual_account= True)

[06/16 00:37:34] INFO MARKET: sync kospi, download https://new.real.download.dws.co.kr/common/master/kospi_code.mst.zip
[06/16 00:37:34] INFO RTC websocket connected
[06/16 00:37:34] INFO MARKET: parseing kospi data... 2328 lines
[06/16 00:37:36] INFO MARKET: sync kosdaq, download https://new.real.download.dws.co.kr/common/master/kosdaq_code.mst.zip
[06/16 00:37:36] INFO MARKET: parseing kosdaq data... 1734 lines
[06/16 00:37:37] INFO MARKET: sync sector, download https://new.real.download.dws.co.kr/common/master/idxcode.mst.zip
[06/16 00:37:37] INFO MARKET: parseing sector data... 477 lines


In [3]:
spark = SparkSession.builder.appName("CreateDataFrameExample").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/16 00:37:41 WARN Utils: Your hostname, DESKTOP-RMO2CFC resolves to a loopback address: 127.0.1.1; using 172.18.9.25 instead (on interface eth0)
24/06/16 00:37:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/16 00:37:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

schema = StructType([
                     StructField('stockCode', StringType(), True)
                    ,StructField('stockName', StringType(), True)
                    ,StructField('priceChange', StringType(), True)
                    ])

data = []
for stock in chain(kis.market.kospi.all(), kis.market.kosdaq.all()) : 
    if (len(stock.mksc_shrn_iscd) == 6) & (stock.prst_cls_code == '0') & (stock.scrt_grp_cls_code == 'ST'): 
        record = {
                    "stockCode"  : stock.mksc_shrn_iscd # 코드 번호 
                    ,"stockName"  : stock.hts_kor_isnm # 코드 이름 
                    ,"priceChange" : stock.fcam_mod_cls_code #  액면가 변경 구분 코드 (00:해당없음 01:액면분할 02:액면병합 99:기타   
                  }
        data.append(record) 

df = spark.createDataFrame(data,schema)
    

In [6]:
def kis_get_values(code,time_from, time_to): 
    time_from = datetime.strptime(time_from, "%Y%m%d")
    time_to = datetime.strptime(time_to, "%Y%m%d")
    # print(code)
    prices = kis.stock(code).period_price(time_from,time_to)
    
    try :         
        #날짜 
        stockdate = [p.stck_bsop_date.strftime("%Y%m%d") for p in prices.prices]
        #종가 
        stockclose = [p.stck_clpr for p in  prices.prices]
        #시가 
        stockopen = [p.stck_oprc for p in  prices.prices]
        #고가
        stockhigh = [p.stck_hgpr for p in  prices.prices]
        #저가
        stocklow = [p.stck_lwpr for p in  prices.prices]
        #거래량
        stockvolume = [p.acml_vol for p in  prices.prices]
        #거래대금
        stockpricevolume = [p.acml_tr_pbmn for p in  prices.prices]    
        return stockdate,stockclose,stockopen,stockhigh,stocklow,stockvolume,stockpricevolume
    
    except : 
        return None

In [8]:
tocker_rdd = df.rdd.map(lambda row: row.stockCode)

In [20]:
kis_get_values(tocker_rdd.take(1)[0],'20240101', '20240614')

[06/14 00:31:28] INFO ACCESS_TOKEN: issue access token


(['20240613',
  '20240612',
  '20240611',
  '20240610',
  '20240607',
  '20240605',
  '20240604',
  '20240603',
  '20240531',
  '20240530',
  '20240529',
  '20240528',
  '20240527',
  '20240524',
  '20240523',
  '20240522',
  '20240521',
  '20240520',
  '20240517',
  '20240516',
  '20240514',
  '20240513',
  '20240510',
  '20240509',
  '20240508',
  '20240507',
  '20240503',
  '20240502',
  '20240430',
  '20240429',
  '20240426',
  '20240425',
  '20240424',
  '20240423',
  '20240422',
  '20240419',
  '20240418',
  '20240417',
  '20240416',
  '20240415',
  '20240412',
  '20240411',
  '20240409',
  '20240408',
  '20240405',
  '20240404',
  '20240403',
  '20240402',
  '20240401',
  '20240329',
  '20240328',
  '20240327',
  '20240326',
  '20240325',
  '20240322',
  '20240321',
  '20240320',
  '20240319',
  '20240318',
  '20240315',
  '20240314',
  '20240313',
  '20240312',
  '20240311',
  '20240308',
  '20240307',
  '20240306',
  '20240305',
  '20240304',
  '20240229',
  '20240228',
  '202

In [1]:
from pykis import *
from itertools import chain
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys
import os
import time
package_path = '/root/project/de-2024/jobs'
sys.path.append(package_path)
import cf
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType

# PyKis 설정
kis = PyKis(
    appkey=cf.appkey,
    appsecret=cf.appsecret,
    virtual_account=True
)

# Spark 세션 생성
conf = SparkConf().setAppName("StockDataApp")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# kis_get_values 함수 정의
def kis_get_values(code, time_from, time_to):
    time_from = datetime.strptime(time_from, "%Y%m%d")
    time_to = datetime.strptime(time_to, "%Y%m%d")
    prices = kis.stock(code).period_price(time_from, time_to)
    try:
        stockdate = [p.stck_bsop_date.strftime("%Y%m%d") for p in prices.prices]
        stockclose = [p.stck_clpr for p in prices.prices]
        stockopen = [p.stck_oprc for p in prices.prices]
        stockhigh = [p.stck_hgpr for p in prices.prices]
        stocklow = [p.stck_lwpr for p in prices.prices]
        stockvolume = [p.acml_vol for p in prices.prices]
        stockpricevolume = [p.acml_tr_pbmn for p in prices.prices]
        return stockdate, stockclose, stockopen, stockhigh, stocklow, stockvolume, stockpricevolume
    except:
        return []

# 날짜 범위 생성 함수
def generate_date_ranges(start_date, end_date, delta_days):
    start_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")
    date_ranges = []
    current_start = start_date
    while current_start < end_date:
        current_end = current_start + timedelta(days=delta_days - 1)
        if current_end > end_date:
            current_end = end_date
        date_ranges.append((current_start.strftime("%Y%m%d"), current_end.strftime("%Y%m%d")))
        current_start = current_end + timedelta(days=1)
    return date_ranges

# 2018년 1월 1일부터 오늘까지 날짜 범위 생성
start_date = "20180101"
end_date = datetime.today().strftime("%Y%m%d")
date_ranges = generate_date_ranges(start_date, end_date, 30)

# 주식 코드 리스트
codes = ["005930"]  # 예시로 주식 코드 리스트 사용

# 날짜 범위와 주식 코드 조합하여 DataFrame 생성
data = [(code, date_range[0], date_range[1]) for code in codes for date_range in date_ranges]
schema = StructType([
    StructField("Code", StringType(), True),
    StructField("TimeFrom", StringType(), True),
    StructField("TimeTo", StringType(), True)
])
stocks_df = spark.createDataFrame(data, schema)

# DataFrame을 RDD로 변환
stock_rdd = stocks_df.rdd.map(lambda row: (row.Code, row.TimeFrom, row.TimeTo))

# kis_get_values 함수를 각 주식 코드와 날짜 범위에 적용
def fetch_stock_data_with_delay(partition):
    from datetime import datetime
    import time
    
    results = []
    for args in partition:
        code, time_from, time_to = args
        try:
            # 딜레이 추가
            time.sleep(1)  # API 호출 제한 시간에 맞게 조정
            stockdate, stockclose, stockopen, stockhigh, stocklow, stockvolume, stockpricevolume = kis_get_values(code, time_from, time_to)
            results.extend(list(zip(stockdate, stockclose, stockopen, stockhigh, stocklow, stockvolume, stockpricevolume, [code] * len(stockdate))))
        except Exception as e:
            print(f"Error fetching data for {code}: {e}")
            continue
    return results

# 주식 데이터를 병렬로 호출하여 RDD로 변환
stock_data_rdd = stock_rdd.mapPartitions(fetch_stock_data_with_delay)

# RDD에서 데이터를 수집하여 단일 리스트로 변환
combined_data = stock_data_rdd.collect()

# DataFrame 스키마 정의
stock_data_schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Close", FloatType(), True),
    StructField("Open", FloatType(), True),
    StructField("High", FloatType(), True),
    StructField("Low", FloatType(), True),
    StructField("Volume", IntegerType(), True),
    StructField("PriceVolume", IntegerType(), True),
    StructField("Code", StringType(), True)
])

# Spark DataFrame으로 변환
stock_data_spark_df = spark.createDataFrame(combined_data, schema=stock_data_schema)

# 데이터 확인
stock_data_spark_df.show()

# Spark 세션 종료
spark.stop()


[06/16 16:15:19] INFO MARKET: up to date kospi
[06/16 16:15:19] INFO MARKET: up to date kosdaq
[06/16 16:15:19] INFO MARKET: up to date sector
[06/16 16:15:19] INFO RTC websocket connected


your 131072x1 screen size is bogus. expect trouble
24/06/16 16:15:20 WARN Utils: Your hostname, DESKTOP-RMO2CFC resolves to a loopback address: 127.0.1.1; using 172.18.9.25 instead (on interface eth0)
24/06/16 16:15:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/16 16:15:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Traceback (most recent call last):
  File "/root/miniconda3/envs/myvenv310/lib/python3.10/site-packages/pyspark/serializers.py", line 459, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/root/miniconda3/envs/myvenv310/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/root/miniconda3/envs/myvenv310/lib/python3.10/site-packages/pyspark/cloudpickle/clou

PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.lock' object

In [22]:
stocks_df.show()

+------+--------+--------+
|  Code|TimeFrom|  TimeTo|
+------+--------+--------+
|005930|20240101|20240130|
|005930|20240131|20240229|
|005930|20240301|20240330|
|005930|20240331|20240429|
|005930|20240430|20240529|
|005930|20240530|20240616|
+------+--------+--------+



In [7]:
kis_get_values('005930', '20240530', '20240616')

AttributeError: module 'pykis.kis' has no attribute 'stock'